In [1]:
import requests

def search_app_store_by_keywords(keywords, country='au'):
    seen_app_ids = set()
    
    for keyword in keywords:
        print(f"Searching for: '{keyword}'")
        url = "https://itunes.apple.com/search"
        params = {
            'term': keyword,
            'country': country,
            'entity': 'software'
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error fetching results for '{keyword}' (status code: {response.status_code})")
            continue

        results = response.json().get('results', [])

        for app in results:
            app_id = app.get('trackId')
            if app_id in seen_app_ids:
                continue  # skip duplicates
            seen_app_ids.add(app_id)

            print(f"Name: {app.get('trackName')}")
            print(f"App ID: {app_id}")
            print(f"Description: {app.get('description', 'No description')[:100]}...")
            print(f"URL: {app.get('trackViewUrl')}")
            print("-" * 40)

# List of keywords
keywords = [
    "distraction", "addiction", "motivation", "self-control",
    "smartphone distraction", "smartphone addiction", "smartphone motivation", "smartphone self-control",
    "laptop distraction", "laptop addiction", "laptop motivation", "laptop self-control",
    "internet distraction", "internet addiction", "internet motivation", "internet self-control"
]

search_app_store_by_keywords(keywords)


Searching for: 'distraction'
Name: Tap 2 Distract
App ID: 6447653942
Description: Tap 2 Distract uses the tool of distraction to encourage play for children during stressful or uncom...
URL: https://apps.apple.com/au/app/tap-2-distract/id6447653942?uo=4
----------------------------------------
Name: Baby Bubbles Babble
App ID: 842788895
Description: When my daughter was about one year old, she loved  bubble popping games. Unfortunately, most of the...
URL: https://apps.apple.com/au/app/baby-bubbles-babble/id842788895?uo=4
----------------------------------------
Name: BlockSite: Block Apps & Focus
App ID: 1474967653
Description: Take control of your time with BlockSite - the ultimate app blocker and site blocker! Stay focused, ...
URL: https://apps.apple.com/au/app/blocksite-block-apps-focus/id1474967653?uo=4
----------------------------------------
Name: Screen Light Color
App ID: 1532469936
Description: Turn your device’s display into a simple light panel with no on-screen distractio

In [2]:
import requests

def fetch_reviews(app_id, country='au'):
    url = f"https://itunes.apple.com/{country}/rss/customerreviews/id={app_id}/json"
    response = requests.get(url)
    if response.status_code != 200:
        print("No data or invalid app ID")
        return

    data = response.json()
    reviews = data.get('feed', {}).get('entry', [])[1:]  # Skip metadata
    for review in reviews:
        print(f"Title: {review['title']['label']}")
        print(f"Rating: {review['im:rating']['label']}")
        print(f"Content: {review['content']['label'][:200]}...\n")

# Example
fetch_reviews(app_id='6447653942')


Title: Best App Ever
Rating: 5
Content: I might be a little biased, but this could be one of the best apps out there that provides anywhere, anytime distraction. The team have worked hard to make this a reality to assist kids with needle ph...

Title: Therapeutic
Rating: 5
Content: Just downloaded this app and couldn’t be happier! Wish I only had this sooner, definitely recommend to all 😊...

Title: This app has been so helpful
Rating: 5
Content: The best for blood tests and needles! Recommend!...



In [5]:
import requests
import time
import csv


def search_app_store_by_keywords(keywords, country='au'):
    seen_app_ids = set()
    apps = []  # Store app info

    for keyword in keywords:
        print(f"\n🔍 Searching for: '{keyword}'")
        url = "https://itunes.apple.com/search"
        params = {
            'term': keyword,
            'country': country,
            'entity': 'software'
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error fetching results for '{keyword}' (status code: {response.status_code})")
            continue

        results = response.json().get('results', [])

        for app in results:
            app_id = app.get('trackId')
            if app_id in seen_app_ids:
                continue  # skip duplicates
            seen_app_ids.add(app_id)

            apps.append({
                'id': app_id,
                'name': app.get('trackName'),
                'url': app.get('trackViewUrl')
            })
    return apps

def fetch_reviews(app_id, country='au'):
    url = f"https://itunes.apple.com/{country}/rss/customerreviews/id={app_id}/json"
    response = requests.get(url)
    if response.status_code != 200:
        return []

    data = response.json()
    entries = data.get('feed', {}).get('entry', [])

    # Handle case: only one review (not a list)
    if isinstance(entries, dict):
        entries = [entries]

    if len(entries) <= 1:
        return []  # Only metadata or no reviews

    return entries[1:]  # Skip metadata

# Define keywords
keywords = [
    "distraction", "addiction", "motivation", "self-control",
    "smartphone distraction", "smartphone addiction", "smartphone motivation", "smartphone self-control",
    "laptop distraction", "laptop addiction", "laptop motivation", "laptop self-control",
    "internet distraction", "internet addiction", "internet motivation", "internet self-control"
]

# Step 1: Search apps
apps = search_app_store_by_keywords(keywords)

# Step 2: Fetch reviews and write to CSV
with open('adhd_matched_apps.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        'App Name', 'App ID', 'Review Title', 'Rating',
        'Review Content', 'Match Location', 'App Description'
    ])

    for app in apps:
        print(f"\n🟡 Checking app: {app['name']} (ID: {app['id']})")

        description = app.get('description', '').strip().replace('\n', ' ')
        description_contains_adhd = 'adhd' in description.lower()

        reviews = fetch_reviews(app['id'])

        # Find reviews that mention ADHD
        matching_reviews = []
        for review in reviews[:5]:  # limit to 5 reviews
            content = review['content']['label'].strip().replace('\n', ' ')
            if 'adhd' in content.lower():
                matching_reviews.append({
                    'title': review['title']['label'],
                    'rating': review['im:rating']['label'],
                    'content': content
                })

        # Only write if ADHD is found in either description or any review
        if description_contains_adhd or matching_reviews:
            match_location = "Both" if (description_contains_adhd and matching_reviews) else (
                "Description" if description_contains_adhd else "Review"
            )

            if matching_reviews:
                for r in matching_reviews:
                    writer.writerow([
                        app['name'], app['id'], r['title'], r['rating'],
                        r['content'], match_location, description
                    ])
            else:
                # No review mentions ADHD, but app description does
                writer.writerow([
                    app['name'], app['id'], '', '', '', match_location, description
                ])
        
        time.sleep(1)



🔍 Searching for: 'distraction'

🔍 Searching for: 'addiction'

🔍 Searching for: 'motivation'

🔍 Searching for: 'self-control'

🔍 Searching for: 'smartphone distraction'

🔍 Searching for: 'smartphone addiction'

🔍 Searching for: 'smartphone motivation'

🔍 Searching for: 'smartphone self-control'

🔍 Searching for: 'laptop distraction'

🔍 Searching for: 'laptop addiction'

🔍 Searching for: 'laptop motivation'

🔍 Searching for: 'laptop self-control'

🔍 Searching for: 'internet distraction'

🔍 Searching for: 'internet addiction'

🔍 Searching for: 'internet motivation'

🔍 Searching for: 'internet self-control'

🟡 Checking app: Tap 2 Distract (ID: 6447653942)

🟡 Checking app: Baby Bubbles Babble (ID: 842788895)

🟡 Checking app: BlockSite: Block Apps & Focus (ID: 1474967653)

🟡 Checking app: Screen Light Color (ID: 1532469936)

🟡 Checking app: Plantie - Stay focused (ID: 1135988868)

🟡 Checking app: HUDWAY Cast — Safe Driving (ID: 1258579266)

🟡 Checking app: Cape: Screen Time Master (ID: 6444